In [1]:
import re
import pyspark
from pyspark.sql import SQLContext


sc = pyspark.SparkContext(appName='hw3-1')
data = sc.wholeTextFiles('data/reut2-*')
newsArticles = data.map(lambda x:x[1]).flatMap(lambda x:x.split('<BODY>')[1:]).map(lambda x:x.split('</BODY>')[0])\
                   .map(lambda x:re.sub(' +', ' ', x.replace('\n', ' ')))

newsArticles.take(3)

['Showers continued throughout the week in the Bahia cocoa zone, alleviating the drought since early January and improving prospects for the coming temporao, although normal humidity levels have not been restored, Comissaria Smith said in its weekly review. The dry period means the temporao will be late this year. Arrivals for the week ended February 22 were 155,221 bags of 60 kilos making a cumulative total for the season of 5.93 mln against 5.81 at the same stage last year. Again it seems that cocoa delivered earlier on consignment was included in the arrivals figures. Comissaria Smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has practically come to an end. With total Bahia crop estimates around 6.4 mln bags and sales standing at almost 6.2 mln there are a few hundred thousand bags still in the hands of farmers, middlemen, exporters and processors. There are doubts as to how much of this cocoa would be fit for export as shippers ar

In [2]:
# (1) Given the Reuters-21578 dataset, please calculate all k-shingles and output the set representation of the text dataset as a matrix.

In [3]:
k = 3
shingles = newsArticles.flatMap(lambda x:[x[i:i+k] for i in range(len(x)-k+1)]).distinct()

shingles.take(5)

['rdk', 'tr.', 'PJ>', 'dop', 'ei-']

In [4]:
shingles_count = shingles.count()
articles_count = newsArticles.count()
print(shingles_count, 'different shingles.')
print(articles_count, 'different articles.')

35077 different shingles.
19043 different articles.


In [5]:
newsArticles = newsArticles.collect()
kShinglesMatrix = shingles.map(lambda s:[1 if s in a else 0 for a in newsArticles])

In [6]:
kShinglesMatrix.coalesce(1).saveAsTextFile('tmp')

In [7]:
!mv /user/bigdata/tmp/part-00000 result/kShinglesMatrix.txt
!rm -rf tmp

mv: cannot stat '/user/bigdata/tmp/part-00000': No such file or directory


In [8]:
# (2) Given the set representation, compute the minhash signatures of all documents

In [9]:

def biggerThanNFirstPrime(N):
    p = 2
    while True:
        isPrime = True
        for i in range(2,p//2+1):
            if(p%i==0):
                isPrime = False
                break
        if isPrime and p > N:
            return p
        else:
            p+=1

In [10]:
import random

h = 100
a = [random.randint(0, 100) for i in range(h)]
b = [random.randint(0, 100) for i in range(h)]
p = biggerThanNFirstPrime(articles_count)
N = articles_count

def rowHash(row, a, b, p, N):
    return ((a*row+b)%p)%N

In [11]:
minHashSignatures = list()
kShinglesMatrixZipWithIndex = kShinglesMatrix.zipWithIndex().cache()
for i in range(h):
    minHashSignatures.append(kShinglesMatrixZipWithIndex\
                             .map(lambda x:[rowHash(x[1], a[i], b[i], p ,N) if c == 1 else (articles_count + 10) for c in x[0]])\
                             .reduce(lambda x, y:[Mx if Mx < My else My for Mx, My in zip(x, y)]))

In [12]:
with open('result/minHashSignatures.txt', 'w') as result:
    for row in minHashSignatures:
        result.write(str(row) + '\n')

In [13]:
import numpy as np
from operator import add
bands = 20
r = 5
similarRate = 0.8
buckets = articles_count
hashFuct = [[random.randint(0, 100) for i in range(r + 1)] for j in range(bands)]

with open('result/candidatePairs.txt', 'w') as result:
    for i in range(articles_count):
        candidatePairs = list()
        for j in range(bands):
            band = np.array(minHashSignatures[j*r:j*r+r]).T
            band = [(np.array(article).dot(np.array(hashFuct[j][:r])) + hashFuct[j][-1]) % buckets for article in band]
            for k, article in enumerate(band):
                if k > i and article == band[i]:
                    candidatePairs.append(k)
        candidatePairs = [(article, candidatePairs.count(article)) for article in set(candidatePairs)]
        candidatePairsTreshold = list()
        for candidatePair in candidatePairs:
            if candidatePair[1] >= bands*similarRate:
                candidatePairsTreshold.append(candidatePair[0])
        result.write('Articles' + str(i) + ':' + str(candidatePairsTreshold) + '\n')

In [14]:
sc.stop()